**https://medium.com/nerd-for-tech/fine-tuning-pretrained-bert-for-sentiment-classification-using-transformers-in-python-931ed142e37t**

In [ ]:
!pip install transformers

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('/content/complaint_data.csv')
df = df.set_index(df.columns[0])
df.head()


In [ ]:
sentences = df['complaint_what_happened'].tolist()
labels = df['Topic'].tolist()

In [4]:
test = pd.DataFrame({
    'complaints': [
        "I can not get from chase who services my mortgage, who owns it and who has original loan docs",
        "The bill amount of my credit card was debited twice. Please look into the matter and resolve at the earliest.",
        "I want to open a salary account at your downtown branch. Please provide me the procedure.",
        "Yesterday, I received a fraudulent email regarding renewal of my services.",
        "What is the procedure to know my CIBIL score?",
        "I need to know the number of bank branches and their locations in the city of Dubai"
    ],
    'labels':[2,1,0,4,0,0]
})

In [5]:
test

,complaints,labels
0,I can not get from chase who services my mortg...,2
1,The bill amount of my credit card was debited ...,1
2,I want to open a salary account at your downto...,0
3,"Yesterday, I received a fraudulent email regar...",4
4,What is the procedure to know my CIBIL score?,0
5,I need to know the number of bank branches and...,0


In [6]:
test_texts = test['complaints'].values.tolist()

In [ ]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(sentences, labels, test_size=.2,random_state=42,stratify=labels)

In [ ]:
import torch
from torch.utils.data import Dataset
from transformers import DistilBertTokenizerFast,DistilBertForSequenceClassification
from transformers import Trainer,TrainingArguments

In [ ]:
model_name  = 'distilbert-base-uncased'

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased',num_labels=5)

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True,return_tensors = 'pt')
val_encodings = tokenizer(val_texts, truncation=True, padding=True,return_tensors = 'pt')
test_encodings = tokenizer(test_texts, truncation=True, padding=True,return_tensors = 'pt')

In [ ]:
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)
## Test Dataset
class SentimentTestDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item
    def __len__(self):
        return len(self.encodings)

In [ ]:
train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)
test_dataset = SentimentTestDataset(test_encodings)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    #recall = recall_score(y_true=labels, y_pred=pred)
    #precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(labels, pred, average='weighted')

    return {"accuracy": accuracy,"f1_score":f1}

In [ ]:
training_args = TrainingArguments(
    output_dir='./res',          # output directory
    evaluation_strategy="steps",
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs4',            # directory for storing logs
    #logging_steps=10,
    load_best_model_at_end=True,
)

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased",num_labels=5)

trainer = Trainer(
    model=model,# the instantiated 🤗 Transformers model to be trained
    args=training_args, # training arguments, defined above
    train_dataset=train_dataset,# training dataset
    eval_dataset=val_dataset , # evaluation dataset
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
trainer.evaluate()

In [8]:
test_label = test['labels'].values.tolist()

In [ ]:
test_encodings = tokenizer(test_texts, truncation=True, padding=True,return_tensors = ‘pt’).to(“cuda”)
test_dataset = SentimentDataset(test_encodings, test_labels)
preds = trainer.predict(test_dataset=test_dataset)

In [ ]:
probs = torch.from_numpy(preds[0]).softmax(1)

predictions = probs.numpy()# convert tensors to numpy array

In [ ]:
newdf = pd.DataFrame(predictions,columns=['Account_Services','Others','Mortgage/Loan','Credit card or prepaid card','Theft/Dispute Reporting'])
new_df.head()

In [ ]:
def labels(x):
  if x == 0:
    return 'Account Services'
  elif x == 1:
    return 'Others'
  elif x == 2:
    return 'Mortgage/Loan'
  elif x == 3:
    return 'Credit card or prepaid card'
  else:
    return 'Theft/Dispute Reporting'

results = np.argmax(predictions,axis=1)
test['complaints'] = results
test['complaints'] = test['complaints'].map(labels)
test.head()

In [ ]:
import seaborn as sns
sns.countplot(x='complaints',data=test)